<a href="https://colab.research.google.com/github/aledima00/Project4_SemSeg_AML2024/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4 - Semantic Segmentation

First let's download dataset, that is already split in "Train", "Test" and "Val" modules

In [1]:
!pip install colorama
!pip install icecream
!pip install albumentations
!pip install yacs
!pip install fvcore
!pip install tqdm

In [2]:
import os
import glob
from skimage.io import imread
import logging
from enum import Enum
import gdown

from icecream import ic

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms

from PIL import Image

import numpy as np

import albumentations

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


General Configuration:

In [3]:
DBG = False                   # set to True for debug mode (lighter execution + dbg prints)
CONFIG_DATASET = True         # set to True to download and config all dataset resources
CONFIG_DEEPLABV2 = True       # set to True to download and config all DeepLabv2 resources
CONFIG_PIDNET = True          # set to True to download and config all PIDNET resources

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
TRAIN_PATH = "Train"          # path of the train folder
VAL_PATH = "Val"              # path of the val folder
TEST_PATH = "Test"            # path of the test folder
MODELS_FOLDER = "saved_models"
if not os.path.isdir(MODELS_FOLDER):
    os.makedirs(MODELS_FOLDER)

DEEPLABV2_WEIGHTS_PATH = "deeplabv2-pretrain-weights.pth"  # path of the deeplabv2 folder
PIDNET_WEIGHTS_PATH = "PIDNet/pretrained_models/imagenet/imagenet.pth.tar"

Using device: cuda


download datasets:

In [4]:
def config_generic_dataset(DS_PATHNAME,URL):
  !rm -rf {DS_PATHNAME}
  ZIP_PATH = DS_PATHNAME + ".zip"
  !rm {ZIP_PATH}
  !wget -O {ZIP_PATH} {URL}
  !unzip {ZIP_PATH} | tail -n 3
  !rm {ZIP_PATH}

def config_train_dataset():
  config_generic_dataset(TRAIN_PATH, "https://zenodo.org/records/5706578/files/Train.zip?download=1")
def config_val_dataset():
  config_generic_dataset(VAL_PATH, "https://zenodo.org/records/5706578/files/Val.zip?download=1")
def config_test_dataset():
  config_generic_dataset(TEST_PATH, "https://zenodo.org/records/5706578/files/Test.zip?download=1")

def config_all_dataset(*,force=False):
  print("Dowloading and Configuring Dataset")
  if force or (not os.path.exists(TRAIN_PATH)):
    config_train_dataset()
  if force or (not os.path.exists(VAL_PATH)):
    config_val_dataset()
  if force or (not os.path.exists(TEST_PATH)):
    config_test_dataset()

download and configure deeplabv2 model library (_with R101 backbone_) and the pretrain weights:

In [5]:
def config_deeplabv2_model():
  print("Dowloading and Configuring DeepLabv2 Model")
  import sys
  import gdown
  !rm -rf "MLDL2024_project1"
  !git clone https://github.com/Gabrysse/MLDL2024_project1.git
  sys.path.append("/content/MLDL2024_project1/")
  gdown.download("https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v", DEEPLABV2_WEIGHTS_PATH, quiet=False)


config pidnet...

In [6]:
def config_pidnet():
  import sys
  import gdown
  print("Dowloading and Configuring PIDNET Model")
  !rm -rf "PIDNet"
  !git clone https://github.com/XuJiacong/PIDNet.git
  sys.path.append("/content/PIDNet/")
  gdown.download("https://drive.google.com/uc?id=1hIBp_8maRr60-B3PF0NVtaA6TYBvO4y-", PIDNET_WEIGHTS_PATH, quiet=False)


In [7]:
if CONFIG_DATASET:
  config_all_dataset()
if CONFIG_DEEPLABV2:
  config_deeplabv2_model()
if CONFIG_PIDNET:
  config_pidnet()

Dowloading and Configuring Dataset
Dowloading and Configuring DeepLabv2 Model
Cloning into 'MLDL2024_project1'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
Receiving objects: 100% (34/34), 12.06 KiB | 12.06 MiB/s, done.
Resolving deltas: 100% (8/8), done.
remote: Total 34 (delta 8), reused 4 (delta 4), pack-reused 16 (from 1)


Downloading...
From (original): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v
From (redirected): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v&confirm=t&uuid=adce1c27-8583-46b3-a437-3f8c3ff7c093
To: /content/deeplabv2-pretrain-weights.pth
100%|██████████| 177M/177M [00:01<00:00, 128MB/s]


Dowloading and Configuring PIDNET Model
Cloning into 'PIDNet'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 386 (delta 131), reused 125 (delta 125), pack-reused 193 (from 1)
Receiving objects: 100% (386/386), 212.80 MiB | 17.74 MiB/s, done.
Resolving deltas: 100% (184/184), done.


Downloading...
From: https://drive.google.com/uc?id=1hIBp_8maRr60-B3PF0NVtaA6TYBvO4y-
To: /content/PIDNet/pretrained_models/imagenet/imagenet.pth.tar
100%|██████████| 38.1M/38.1M [00:00<00:00, 57.7MB/s]


In [8]:
from colorama import Fore, Back, Style
def dbgp(name,value):
  """ Debug print function """
  if DBG:
    print(f"{Fore.BLACK}{Back.GREEN}{Style.BRIGHT}{name}:\t{value}{Fore.RESET}{Back.RESET}{Style.RESET_ALL}")

## Data Preparation

Create Dataset class and filter urban pictures...

In [9]:
NUM_CLASSES = 7
BATCH_SIZE = 2 if DBG else 16
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STDDEV = (0.229, 0.224, 0.225)

In [10]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transform = A.Compose([
      A.Resize(256,256),
      A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STDDEV),
      ToTensorV2()
  ])

test_transform = A.Compose([
    A.Resize(256,256),
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STDDEV),
    ToTensorV2()
])


In [11]:
# taken from official repo of LoveDA
IGNORE_INDEX = -1
COLOR_MAP = {
    IGNORE_INDEX:"IGNORE",
    0:"Background",
    1:"Building",
    2:"Road",
    3:"Water",
    4:"Barren",
    5:"Forest",
    6:"Agricultural"
}
CLASSES = list(key for key in COLOR_MAP.keys() if COLOR_MAP[key] != "IGNORE")

In [12]:
def pil_loader(path,*,format:str="RGB"):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert(format)

class DataType(Enum):
  RURAL = 0
  URBAN = 1

class LoveDA(Dataset):
  def __init__(self, basedir, data_type:DataType, transforms=None):
    #super(LoveDA, self).__init__(basedir, transforms, target_transform) # should we do this??
    if data_type == DataType.RURAL:
        self.base_path = os.path.join(basedir, "Rural")
    else: #data_type == DataType.URBAN:
        self.base_path = os.path.join(basedir, "Urban")


    # list of integers that identifies paths relative to both images_png and masks_png
    self.int_pathrefs = os.listdir(os.path.join(self.base_path, "images_png"))
    self.int_pathrefs = list(int(st.split(".")[0]) for st in self.int_pathrefs)

    # DEBUG PRINT
    if DBG:
      self.int_pathrefs = self.int_pathrefs[:15] # limit the dataset for debug

    self.transforms = transforms

  def __getitem__(self, idx):
    assert idx < len(self), 'Index out of range'
    image_path = os.path.join(self.base_path, "images_png", str(self.int_pathrefs[idx]) + ".png")
    mask_path = os.path.join(self.base_path, "masks_png", str(self.int_pathrefs[idx]) + ".png")
    image = pil_loader(image_path,format="RGB")
    mask = pil_loader(mask_path,format="L")

    # Convert PIL images to numpy arrays
    image = np.array(image)
    mask = np.array(mask, dtype=np.int8)

    if self.transforms is not None:
      augmented = self.transforms(image=image, mask=mask)
      image = augmented["image"]
      mask = augmented["mask"]

    mask -= 1

    return image, mask

  def __len__(self):
    return len(self.int_pathrefs)


In [13]:
# Urban Datasets (train, val, test)
urban_train = LoveDA(TRAIN_PATH, DataType.URBAN, transforms=train_transform)
urban_val = LoveDA(VAL_PATH, DataType.URBAN, transforms=test_transform)

# Urban Dataloaders (train, val, test)

NUM_WORKERS = 2 if DBG else 4
urban_train_dataloader = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
urban_val_dataloader = DataLoader(urban_val, shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


### Analysis Utility

In [14]:
from fvcore.nn import FlopCountAnalysis, flop_count_table
import time
from tqdm.auto import tqdm

def analyze_model(model,*,iterations=100,batch_size=1):

  model.training=False
  height = 256
  width = 256
  image = torch.zeros((batch_size,3, height, width)).to(DEVICE)

  # Set the model to evaluation mode to avoid issues with batch normalization
  model.eval()

  flops = FlopCountAnalysis(model, image)
  print(flop_count_table(flops))

  # latency compute
  latency = list()
  start_event = torch.cuda.Event(enable_timing=True)
  end_event = torch.cuda.Event(enable_timing=True)
  for _ in tqdm(range(iterations)):
    start_event.record()  # Record start time on GPU
    _ = model(image)      # Run inference
    end_event.record()    # Record end time on GPU

    # Wait for GPU synchronization to ensure accurate timing
    torch.cuda.synchronize()

    # Calculate time elapsed in milliseconds
    latency.append(start_event.elapsed_time(end_event))
  avg_latency = sum(latency) / len(latency)
  print(f"Average latency: {avg_latency:.2f} ms")

## DeepLabv2 on LoveDA (Urban)

In [15]:
LR = 2e-4               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-4     # Regularization, you can keep this at the default
NUM_EPOCHS = 20         # Total number of training epochs (iterations over dataset)
STEP_SIZE = [10, 15]    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

Get DeepLabv2 model with pretrain weights:

In [16]:
from MLDL2024_project1.models.deeplabv2 import deeplabv2

model = deeplabv2.get_deeplab_v2(num_classes=NUM_CLASSES,pretrain=True,pretrain_model_path=DEEPLABV2_WEIGHTS_PATH)
model_name = "deeplab_v2"

Deeplab pretraining loading...


/content/MLDL2024_project1/models/deeplabv2/deeplabv2.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(pretrain_model_path)


### Optimizer, Loss, ecc.

In [17]:
# enable validation during training
validate = True

model = model.to(DEVICE) # switch to GPU

# Loss (as said in DeepLabv2 docs)
loss_fn = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

# Opt
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer_fn = optim.Adam(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-4)

#Scheduler
optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=STEP_SIZE, gamma=GAMMA)

In [18]:
analyze_model(model,iterations=1000,batch_size=1)

| module                         | #parameters or shape   | #flops     |
|:-------------------------------|:-----------------------|:-----------|
| model                          | 43.016M                | 47.669G    |
|  conv1                         |  9.408K                |  0.154G    |
|   conv1.weight                 |   (64, 3, 7, 7)        |            |
|  bn1                           |  0.128K                |  2.097M    |
|   bn1.weight                   |   (64,)                |            |
|   bn1.bias                     |   (64,)                |            |
|  layer1                        |  0.216M                |  0.912G    |
|   layer1.0                     |   75.008K              |   0.317G   |
|    layer1.0.conv1              |    4.096K              |    17.306M |
|    layer1.0.bn1                |    0.128K              |    0.541M  |
|    layer1.0.conv2              |    36.864K             |    0.156G  |
|    layer1.0.bn2                |    0.128K       

  0%|          | 0/1000 [00:00<?, ?it/s]

Average latency: 35.57 ms


### Training Loop

In [19]:
import warnings
from torch.backends import cudnn
warnings.filterwarnings('ignore')

train_iter = 0
iterPerEpoch = len(urban_train_dataloader)

cudnn.benchmark
CUDA_LAUNCH_BLOCKING=1

best_IoU = 0

EPSILON_IOU = 1e-7

for epoch in range(NUM_EPOCHS):
    model.train(True)
    epoch_loss = 0

    total_intersection_per_class = {cls: 0 for cls in CLASSES}
    total_union_per_class = {cls: 0 for cls in CLASSES}

    for i, (inputs, targets) in enumerate(urban_train_dataloader):

        optimizer_fn.zero_grad()

        # feeds in model
        inputs = inputs.to(DEVICE)
        labels = targets.long().to(device=DEVICE)
        output_logits,_,_ = model(inputs)

        # compute loss
        loss = loss_fn(output_logits, labels)

        # backward loss and optimizer step
        loss.backward()
        optimizer_fn.step()

        #compute the training accuracy
        _, predicted = torch.max(output_logits.data, 1)

        for cls in CLASSES:
            true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
            pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

            # Calcola l'intersezione e l'unione per quella classe
            intersection = torch.logical_and(true_mask, pred_mask).sum().item()
            union = torch.logical_or(true_mask, pred_mask).sum().item()

            # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
            total_intersection_per_class[cls] += intersection
            total_union_per_class[cls] += union

        step_loss = loss.data.item()
        epoch_loss += step_loss

    # FINAL EPOCH-WISE COMPUTATIONS
    class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
    mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
    avg_loss = epoch_loss/iterPerEpoch
    print(Fore.GREEN + Style.NORMAL + 'Train: Epoch = {} | mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch + 1, avg_loss, mean_IoU)+Style.RESET_ALL)

    if validate:
        iterPerVal = len(urban_val_dataloader)
        model.eval()
        val_loss = 0
        numCorr = 0
        total_intersection_per_class = {cls: 0 for cls in CLASSES}
        total_union_per_class = {cls: 0 for cls in CLASSES}
        for j, (inputs, targets) in enumerate(urban_val_dataloader):

            # feeds in model
            inputs = inputs.to(DEVICE)
            labels = targets.long().to(device=DEVICE)
            output_logits = model(inputs)

            # compute loss
            loss = loss_fn(output_logits, labels)

            # compute the training accuracy
            _, predicted = torch.max(output_logits.data, 1)

            for cls in CLASSES:
                true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
                pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

                # Calcola l'intersezione e l'unione per quella classe
                intersection = torch.logical_and(true_mask, pred_mask).sum().item()
                union = torch.logical_or(true_mask, pred_mask).sum().item()

                # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
                total_intersection_per_class[cls] += intersection
                total_union_per_class[cls] += union

            step_loss = loss.data.item()
            val_loss += step_loss


        # FINAL EPOCH-WISE COMPUTATIONS
        class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
        mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
        avg_loss = val_loss/iterPerVal
        print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'VALIDATION RESULTS (@epoch={}): mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)
        # print IoU for each class
        print(Fore.CYAN + Style.NORMAL + "Class-wise IoUs:"+ Style.RESET_ALL)
        for cls in CLASSES:
            print(Fore.WHITE + Style.DIM + f"Class {cls} ({COLOR_MAP[cls]}): IoU = {class_IoUs[cls]:.3f}"+ Style.RESET_ALL)

        if mean_IoU > best_IoU:
            best_IoU = mean_IoU
            save_path = os.path.join(MODELS_FOLDER, model_name)
            print(f"New Best Model! Saving it to {save_path}")
            torch.save(model.state_dict(), save_path)
        # END OF VALIDATION

    optim_scheduler.step()


Train: Epoch = 1 | mean Loss = 1.331 | mean-IoU = 0.248
VALIDATION RESULTS (@epoch=1): mean Loss = 2.192 | mean-IoU = 0.192
Class-wise IoUs:
Class 0 (Background): IoU = 0.236
Class 1 (Building): IoU = 0.300
Class 2 (Road): IoU = 0.372
Class 3 (Water): IoU = 0.323
Class 4 (Barren): IoU = 0.107
Class 5 (Forest): IoU = 0.002
Class 6 (Agricultural): IoU = 0.007
New Best Model! Saving it to saved_models/deeplab_v2
Train: Epoch = 2 | mean Loss = 0.826 | mean-IoU = 0.461
VALIDATION RESULTS (@epoch=2): mean Loss = 5.778 | mean-IoU = 0.321
Class-wise IoUs:
Class 0 (Background): IoU = 0.315
Class 1 (Building): IoU = 0.355
Class 2 (Road): IoU = 0.424
Class 3 (Water): IoU = 0.437
Class 4 (Barren): IoU = 0.229
Class 5 (Forest): IoU = 0.299
Class 6 (Agricultural): IoU = 0.191
New Best Model! Saving it to saved_models/deeplab_v2
Train: Epoch = 3 | mean Loss = 0.661 | mean-IoU = 0.547
VALIDATION RESULTS (@epoch=3): mean Loss = 1.554 | mean-IoU = 0.342
Class-wise IoUs:
Class 0 (Background): IoU = 0.276

Now we test using the val dataset:

In [20]:
model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, model_name)))
model.eval()
iterPerVal = len(urban_val_dataloader)
model.eval()
val_loss = 0
numCorr = 0
total_intersection_per_class = {cls: 0 for cls in CLASSES}
total_union_per_class = {cls: 0 for cls in CLASSES}
for j, (inputs, targets) in enumerate(urban_val_dataloader):

    # feeds in model
    inputs = inputs.to(DEVICE)
    labels = targets.long().to(device=DEVICE)
    output_logits = model(inputs)

    # compute loss
    loss = loss_fn(output_logits, labels)

    # compute the training accuracy
    _, predicted = torch.max(output_logits.data, 1)

    for cls in CLASSES:
        true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
        pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

        # Calcola l'intersezione e l'unione per quella classe
        intersection = torch.logical_and(true_mask, pred_mask).sum().item()
        union = torch.logical_or(true_mask, pred_mask).sum().item()

        # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
        total_intersection_per_class[cls] += intersection
        total_union_per_class[cls] += union

    step_loss = loss.data.item()
    val_loss += step_loss


# FINAL EPOCH-WISE COMPUTATIONS
class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
avg_loss = val_loss/iterPerVal
print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'TEST RESULTS: mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)
# print IoU for each class
print(Fore.CYAN + Style.NORMAL + "Class-wise IoUs:"+ Style.RESET_ALL)
for cls in CLASSES:
    print(Fore.WHITE + Style.DIM + f"Class {cls} ({COLOR_MAP[cls]}): IoU = {class_IoUs[cls]:.3f}"+ Style.RESET_ALL)

TEST RESULTS: mean Loss = 20.000 | mean-IoU = 1.678
Class-wise IoUs:
Class 0 (Background): IoU = 0.299
Class 1 (Building): IoU = 0.406
Class 2 (Road): IoU = 0.453
Class 3 (Water): IoU = 0.608
Class 4 (Barren): IoU = 0.246
Class 5 (Forest): IoU = 0.321
Class 6 (Agricultural): IoU = 0.283


## PIDNet & LoveDA

In [21]:
### copied from repo ###

def load_pretrained(model, pretrained):
    pretrained_dict = torch.load(pretrained, map_location='cpu')
    if 'state_dict' in pretrained_dict:
        pretrained_dict = pretrained_dict['state_dict']
    model_dict = model.state_dict()
    #pretrained_dict = {k[6:]: v for k, v in pretrained_dict.items() if (k[6:] in model_dict and v.shape == model_dict[k[6:]].shape)}
    msg = 'Loaded {} parameters!'.format(len(pretrained_dict))
    print('Attention!!!')
    print(msg)
    print('Over!!!')
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict, strict = False)

    return model
########################

In [22]:
from PIDNet.models.pidnet import PIDNet
model = PIDNet(m=2, n=3, num_classes=NUM_CLASSES, planes=32, ppm_planes=96, head_planes=128, augment=True)
model = load_pretrained(model, PIDNET_WEIGHTS_PATH)
model_name = "PIDNet"

Attention!!!
Loaded 322 parameters!
Over!!!


### Optimizer, Loss, ecc.

In [23]:
LR = 2e-4               # The initial Learning Rate
MOMENTUM = 0.9          # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-4     # Regularization, you can keep this at the default
NUM_EPOCHS = 20         # Total number of training epochs (iterations over dataset)
STEP_SIZE = [10, 15]    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1             # Multiplicative factor for learning rate step-down

In [24]:
# enable validation during training
validate = True

model.train()
model = model.to(DEVICE) # switch to GPU

# loss functions
sem_criterion = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)

#Opt
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer_fn = optim.Adam(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-4)

#Scheduler
optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=STEP_SIZE, gamma=GAMMA)

# model = FullModel(model, sem_loss=sem_criterion, bd_loss=bd_criterion)


In [25]:
analyze_model(model,iterations=1000,batch_size=1)

| module                      | #parameters or shape   | #flops     |
|:----------------------------|:-----------------------|:-----------|
| model                       | 7.718M                 | 1.579G     |
|  conv1                      |  10.272K               |  53.215M   |
|   conv1.0                   |   0.896K               |   14.156M  |
|    conv1.0.weight           |    (32, 3, 3, 3)       |            |
|    conv1.0.bias             |    (32,)               |            |
|   conv1.1                   |   64                   |   1.049M   |
|    conv1.1.weight           |    (32,)               |            |
|    conv1.1.bias             |    (32,)               |            |
|   conv1.3                   |   9.248K               |   37.749M  |
|    conv1.3.weight           |    (32, 32, 3, 3)      |            |
|    conv1.3.bias             |    (32,)               |            |
|   conv1.4                   |   64                   |   0.262M   |
|    conv1.4.weight 

  0%|          | 0/1000 [00:00<?, ?it/s]

Average latency: 15.36 ms


### Training loop

In [ ]:
import warnings

from torch.backends import cudnn
warnings.filterwarnings('ignore')

iterPerEpoch = len(urban_train_dataloader)

cudnn.benchmark
CUDA_LAUNCH_BLOCKING=1

best_IoU = 0
EPSILON_IOU = 1e-7

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0
    total_intersection_per_class = {cls: 0 for cls in CLASSES}
    total_union_per_class = {cls: 0 for cls in CLASSES}

    for i, (inputs, targets) in enumerate(urban_train_dataloader):
        optimizer_fn.zero_grad()

        # send inputs to gpu
        inputs = inputs.to(DEVICE)
        labels = targets.long().to(device=DEVICE)

        # feeds in the model
        output_logits,_,_ = model(inputs)

        h, w = labels.size(1), labels.size(2)
        ph, pw = output_logits.size(2), output_logits.size(3)
        if ph != h or pw != w:
          output_logits = F.interpolate(output_logits, size=(h, w), mode='bilinear', align_corners=True)


        # compute loss
        loss = sem_criterion(output_logits, labels)
        """
        filler = torch.ones_like(labels) * config.TRAIN.IGNORE_LABEL
        bd_label = torch.where(F.sigmoid(outputs[-1][:,0,:,:])>0.8, labels, filler)
        loss_sb = self.sem_loss(outputs[-2], bd_label)
        loss += loss_sb
        """

        # backward loss and optimizer step
        loss.backward()
        optimizer_fn.step()

        #compute the training accuracy
        _, predicted = torch.max(output_logits.data, 1)

        for cls in CLASSES:
            true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
            pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

            # Calcola l'intersezione e l'unione per quella classe
            intersection = torch.logical_and(true_mask, pred_mask).sum().item()
            union = torch.logical_or(true_mask, pred_mask).sum().item()

            # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
            total_intersection_per_class[cls] += intersection
            total_union_per_class[cls] += union

        step_loss = loss.data.item()
        epoch_loss += step_loss

    # FINAL EPOCH-WISE COMPUTATIONS
    class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
    mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
    avg_loss = epoch_loss/iterPerEpoch
    print(Fore.GREEN + Style.NORMAL + 'Train: Epoch = {} | mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch + 1, avg_loss, mean_IoU)+Style.RESET_ALL)

    # Stampa l'IoU per ogni classe
    print(Fore.CYAN + Style.NORMAL + "Class-wise IoUs:"+ Style.RESET_ALL)
    for cls in CLASSES:
        print(Fore.WHITE + Style.DIM + f"Class {cls} ({COLOR_MAP[cls]}): IoU = {class_IoUs[cls]:.3f}"+ Style.RESET_ALL)

    if validate:
        model.eval()
        val_loss = 0
        iterPerVal = len(urban_val_dataloader)
        numCorr = 0
        total_intersection_per_class = {cls: 0 for cls in CLASSES}
        total_union_per_class = {cls: 0 for cls in CLASSES}
        for j, (inputs, targets) in enumerate(urban_val_dataloader):

            # feeds in model
            inputs = inputs.to(DEVICE)
            labels = targets.long().to(device=DEVICE)
            output_logits,_,_ = model(inputs)
            output_logits = F.interpolate(output_logits, size=labels.shape[1:], mode='bilinear', align_corners=True)

            # compute loss
            loss = sem_criterion(output_logits, labels)


            # compute the training accuracy
            _, predicted = torch.max(output_logits.data, 1)

            for cls in CLASSES:
                true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
                pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

                # Calcola l'intersezione e l'unione per quella classe
                intersection = torch.logical_and(true_mask, pred_mask).sum().item()
                union = torch.logical_or(true_mask, pred_mask).sum().item()

                # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
                total_intersection_per_class[cls] += intersection
                total_union_per_class[cls] += union

            step_loss = loss.data.item()
            val_loss += step_loss


        # FINAL EPOCH-WISE COMPUTATIONS
        class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
        mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
        avg_loss = val_loss/iterPerVal
        print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'VALIDATION RESULTS (@epoch={}): mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)

        if mean_IoU > best_IoU:
            best_IoU = mean_IoU
            save_path = os.path.join(MODELS_FOLDER, model_name)
            print(f"New Best Model! Saving it to {save_path}")
            torch.save(model.state_dict(), save_path)
        # END OF VALIDATION

    optim_scheduler.step()

Train: Epoch = 1 | mean Loss = 1.950 | mean-IoU = 0.207
Class-wise IoUs:
Class 0 (Background): IoU = 0.378
Class 1 (Building): IoU = 0.285
Class 2 (Road): IoU = 0.131
Class 3 (Water): IoU = 0.249
Class 4 (Barren): IoU = 0.168
Class 5 (Forest): IoU = 0.221
Class 6 (Agricultural): IoU = 0.014
VALIDATION RESULTS (@epoch=1): mean Loss = 1.996 | mean-IoU = 0.233
New Best Model! Saving it to saved_models/PIDNet
Train: Epoch = 2 | mean Loss = 1.089 | mean-IoU = 0.306
Class-wise IoUs:
Class 0 (Background): IoU = 0.488
Class 1 (Building): IoU = 0.357
Class 2 (Road): IoU = 0.259
Class 3 (Water): IoU = 0.422
Class 4 (Barren): IoU = 0.256
Class 5 (Forest): IoU = 0.325
Class 6 (Agricultural): IoU = 0.037
VALIDATION RESULTS (@epoch=2): mean Loss = 1.778 | mean-IoU = 0.261
New Best Model! Saving it to saved_models/PIDNet
Train: Epoch = 3 | mean Loss = 0.957 | mean-IoU = 0.362
Class-wise IoUs:
Class 0 (Background): IoU = 0.514
Class 1 (Building): IoU = 0.398
Class 2 (Road): IoU = 0.319
Class 3 (Water)

In [ ]:
model.eval()
val_loss = 0
iterPerVal = len(urban_val_dataloader)
numCorr = 0
total_intersection_per_class = {cls: 0 for cls in CLASSES}
total_union_per_class = {cls: 0 for cls in CLASSES}
for j, (inputs, targets) in enumerate(urban_val_dataloader):

    # feeds in model
    inputs = inputs.to(DEVICE)
    labels = targets.long().to(device=DEVICE)
    output_logits,_,_ = model(inputs)
    output_logits = F.interpolate(output_logits, size=labels.shape[1:], mode='bilinear', align_corners=True)

    # compute loss
    loss = sem_criterion(output_logits, labels)


    # compute the training accuracy
    _, predicted = torch.max(output_logits.data, 1)

    for cls in CLASSES:
        true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
        pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

        # Calcola l'intersezione e l'unione per quella classe
        intersection = torch.logical_and(true_mask, pred_mask).sum().item()
        union = torch.logical_or(true_mask, pred_mask).sum().item()

        # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
        total_intersection_per_class[cls] += intersection
        total_union_per_class[cls] += union

    step_loss = loss.data.item()
    val_loss += step_loss


# FINAL EPOCH-WISE COMPUTATIONS
class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
avg_loss = val_loss/iterPerVal
print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'VALIDATION RESULTS (@epoch={}): mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)

if mean_IoU > best_IoU:
    best_IoU = mean_IoU
    save_path = os.path.join(MODELS_FOLDER, model_name)
    print(f"New Best Model! Saving it to {save_path}")
    torch.save(model.state_dict(), save_path)
# END OF VALIDATION

## Domain Shift

Add dataset and loaders for rural images:

In [ ]:
# Rural Datasets (train, val, test)
rural_val = LoveDA(VAL_PATH, DataType.RURAL, transforms=test_transform)

# Urban Dataloaders (train, val, test)

NUM_WORKERS = 2 if DBG else 4
rural_val_dataloader = DataLoader(rural_val, shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

first let's test with domain shift:

In [ ]:
model.eval()
val_loss = 0
iterPerVal = len(rural_val_dataloader)
numCorr = 0
total_intersection_per_class = {cls: 0 for cls in CLASSES}
total_union_per_class = {cls: 0 for cls in CLASSES}
for j, (inputs, targets) in enumerate(rural_val_dataloader):

    # feeds in model
    inputs = inputs.to(DEVICE)
    labels = targets.long().to(device=DEVICE)
    output_logits,_,_ = model(inputs)
    output_logits = F.interpolate(output_logits, size=labels.shape[1:], mode='bilinear', align_corners=True)

    # compute loss
    loss = sem_criterion(output_logits, labels)


    # compute the training accuracy
    _, predicted = torch.max(output_logits.data, 1)

    for cls in CLASSES:
        true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
        pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

        # Calcola l'intersezione e l'unione per quella classe
        intersection = torch.logical_and(true_mask, pred_mask).sum().item()
        union = torch.logical_or(true_mask, pred_mask).sum().item()

        # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
        total_intersection_per_class[cls] += intersection
        total_union_per_class[cls] += union

    step_loss = loss.data.item()
    val_loss += step_loss


# FINAL EPOCH-WISE COMPUTATIONS
class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
avg_loss = val_loss/iterPerVal
print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'VALIDATION RESULTS (@epoch={}): mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)

if mean_IoU > best_IoU:
    best_IoU = mean_IoU
    save_path = os.path.join(MODELS_FOLDER, model_name)
    print(f"New Best Model! Saving it to {save_path}")
    torch.save(model.state_dict(), save_path)
# END OF VALIDATION

In [ ]:
aug1 = A.Compose([
          A.HorizontalFlip(),
          A.RandomRotate90(),
          A.ColorJitter()
          ],p= 0.5)
aug2 = A.Compose([
          A.RandomBrightnessContrast(),
          A.RandomGamma(),
          A.GaussianBlur(),
          ],p= 0.5)
aug3 = A.Compose([
          A.HorizontalFlip(),
          A.RandomRotate90(),
          A.ColorJitter(),
          A.RandomBrightnessContrast(),
          A.RandomGamma(),
          A.GaussianBlur(),
          ],p= 0.5)

augmentations = [aug1,aug2,aug3]


train_transform_augmentations =
[
  A.Compose([
      A.Resize(256,256),
      aug_i,
      A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STDDEV),
      ToTensorV2()
  ])
  for aug_i in augmentations
]


In [ ]:
for i, trf in enumerate(train_transform_augmentations):

    # Urban Datasets (train, val, test)
    urban_train_augmented = LoveDA(TRAIN_PATH, DataType.URBAN, transforms=trf)
    rural_val = LoveDA(VAL_PATH, DataType.RURAL, transforms=test_transform)

    # Urban Dataloaders (train, val, test)

    NUM_WORKERS = 2 if DBG else 4
    urban_train_dataloader_augmented = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
    urban_val_dataloader = DataLoader(rural_val, shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

    import warnings

    from torch.backends import cudnn
    warnings.filterwarnings('ignore')

    iterPerEpoch = len(urban_train_dataloader)

    cudnn.benchmark
    CUDA_LAUNCH_BLOCKING=1

    best_IoU = 0
    EPSILON_IOU = 1e-7

    for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = 0
        total_intersection_per_class = {cls: 0 for cls in CLASSES}
        total_union_per_class = {cls: 0 for cls in CLASSES}

        for i, (inputs, targets) in enumerate(urban_train_dataloader):
            optimizer_fn.zero_grad()

            # send inputs to gpu
            inputs = inputs.to(DEVICE)
            labels = targets.long().to(device=DEVICE)

            # feeds in the model
            output_logits,_,_ = model(inputs)

            h, w = labels.size(1), labels.size(2)
            ph, pw = output_logits.size(2), output_logits.size(3)
            if ph != h or pw != w:
              output_logits = F.interpolate(output_logits, size=(h, w), mode='bilinear', align_corners=True)


            # compute loss
            loss = sem_criterion(output_logits, labels)
            """
            filler = torch.ones_like(labels) * config.TRAIN.IGNORE_LABEL
            bd_label = torch.where(F.sigmoid(outputs[-1][:,0,:,:])>0.8, labels, filler)
            loss_sb = self.sem_loss(outputs[-2], bd_label)
            loss += loss_sb
            """

            # backward loss and optimizer step
            loss.backward()
            optimizer_fn.step()

            #compute the training accuracy
            _, predicted = torch.max(output_logits.data, 1)

            for cls in CLASSES:
                true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
                pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

                # Calcola l'intersezione e l'unione per quella classe
                intersection = torch.logical_and(true_mask, pred_mask).sum().item()
                union = torch.logical_or(true_mask, pred_mask).sum().item()

                # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
                total_intersection_per_class[cls] += intersection
                total_union_per_class[cls] += union

            step_loss = loss.data.item()
            epoch_loss += step_loss

        # FINAL EPOCH-WISE COMPUTATIONS
        class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
        mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
        avg_loss = epoch_loss/iterPerEpoch
        print(Fore.GREEN + Style.NORMAL + 'Train: Epoch = {} | mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch + 1, avg_loss, mean_IoU)+Style.RESET_ALL)

        # Stampa l'IoU per ogni classe
        print(Fore.CYAN + Style.NORMAL + "Class-wise IoUs:"+ Style.RESET_ALL)
        for cls in CLASSES:
            print(Fore.WHITE + Style.DIM + f"Class {cls} ({COLOR_MAP[cls]}): IoU = {class_IoUs[cls]:.3f}"+ Style.RESET_ALL)

        if validate:
            model.eval()
            val_loss = 0
            iterPerVal = len(rural_val_dataloader)
            numCorr = 0
            total_intersection_per_class = {cls: 0 for cls in CLASSES}
            total_union_per_class = {cls: 0 for cls in CLASSES}
            for j, (inputs, targets) in enumerate(rural_val_dataloader):

                # feeds in model
                inputs = inputs.to(DEVICE)
                labels = targets.long().to(device=DEVICE)
                output_logits,_,_ = model(inputs)
                output_logits = F.interpolate(output_logits, size=labels.shape[1:], mode='bilinear', align_corners=True)

                # compute loss
                loss = sem_criterion(output_logits, labels)


                # compute the training accuracy
                _, predicted = torch.max(output_logits.data, 1)

                for cls in CLASSES:
                    true_mask = (labels == cls)  # Crea una maschera booleana per la classe `cls` nel target
                    pred_mask = (predicted == cls)  # Crea una maschera booleana per la classe `cls` nelle predizioni

                    # Calcola l'intersezione e l'unione per quella classe
                    intersection = torch.logical_and(true_mask, pred_mask).sum().item()
                    union = torch.logical_or(true_mask, pred_mask).sum().item()

                    # Aggiungi i valori all'array totale di intersezione e unione per ogni classe
                    total_intersection_per_class[cls] += intersection
                    total_union_per_class[cls] += union

                step_loss = loss.data.item()
                val_loss += step_loss


            # FINAL EPOCH-WISE COMPUTATIONS
            class_IoUs = { cls: total_intersection_per_class[cls] / (total_union_per_class[cls] + EPSILON_IOU) for cls in CLASSES }
            mean_IoU = sum(class_IoUs.values()) / NUM_CLASSES
            avg_loss = val_loss/iterPerVal
            print(Fore.BLACK + Back.YELLOW + Style.BRIGHT + 'VALIDATION RESULTS (@epoch={}): mean Loss = {:.3f} | mean-IoU = {:.3f}'.format(epoch+1, avg_loss, mean_IoU)+Style.RESET_ALL)

            if mean_IoU > best_IoU:
                best_IoU = mean_IoU
                save_path = os.path.join(MODELS_FOLDER, model_name)
                print(f"New Best Model! Saving it to {save_path}")
                torch.save(model.state_dict(), save_path)
            # END OF VALIDATION

        optim_scheduler.step()


## 5

In [ ]:

import cv2
def canny_with_cv2(images_tensors, low_threshold=0.1, high_threshold=0.2):
    # Convert to NumPy
    edges_tensors = images_tensors.clone().cpu()
    for i,img in enumerate(edges_tensors):
      # Convert to NumPy
      image_np = img.numpy()
      edges_np = cv2.Canny((image_np*255).astype('uint8'), low_threshold, high_threshold)

      # Convert back to tensor
      edges_tensors[i] = torch.from_numpy(edges_np).float() / 255.0
    return edges_tensors